## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Package Loading](#package)
* [Model selection](#model_selection)
    * [FD001](#fd001)
    * [FD002](#fd002)
    * [FD003](#fd003)
    * [FD004](#fd004)

## Package's loading <a id='package'></a>

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
import tensorflow as tf
import seaborn as sns
import sklearn
import random
import math
import time



from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from tensorflow.keras import optimizers


from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
from methods import *

%matplotlib inline
warnings.filterwarnings('ignore')

SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

### FD004 <a class="anchor" id="fd004"></a>

In [2]:
# Load data
# Load data and preprocess
with tf.device('/device:GPU:0'):
    train, test, y_test = prepare_data('FD004.txt')
    print(train.shape, test.shape, y_test.shape)
    sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

    remaining_sensors = ["Ps30", "Nf", "phi", "BPR", "farB"] # selection based on main_notebook

    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 120
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(61249, 27) (41214, 26) (248, 1)


In [3]:
# 1layers
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(LSTM(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    # model.save_weights(weights_file)

    return model

In [4]:
%%time

# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	nodes	dropout	activation	batch_size	TW
# 1	15.449126	0.0	1.242658e+03	0.0	238.675491	0.0	[64]	0.2	tanh	32	40
# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 2	16.100150	0.0	1.752538e+03	0.0	259.214813	0.0	0.30	[64]	0.2	tanh	32	40
results = pd.DataFrame()

for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.3
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)
    results.to_csv('results/base/fd004.csv')

(51538, 40, 5) (51538, 1) (248, 40, 5)
Epoch 1/20
1611/1611 [==============================] - 11s 7ms/step - loss: 1007.3642 - val_loss: 1328.0765
Epoch 2/20
1611/1611 [==============================] - 10s 6ms/step - loss: 455.5491 - val_loss: 903.4106
Epoch 3/20
1611/1611 [==============================] - 10s 7ms/step - loss: 363.3373 - val_loss: 509.7916
Epoch 4/20
1611/1611 [==============================] - 10s 6ms/step - loss: 314.2787 - val_loss: 459.2633
Epoch 5/20
1611/1611 [==============================] - 11s 7ms/step - loss: 298.5221 - val_loss: 375.7044
Epoch 6/20
1611/1611 [==============================] - 11s 7ms/step - loss: 279.5657 - val_loss: 393.4356
Epoch 7/20
1611/1611 [==============================] - 11s 7ms/step - loss: 259.8807 - val_loss: 302.1973
Epoch 8/20
1611/1611 [==============================] - 11s 7ms/step - loss: 243.9541 - val_loss: 319.0983
Epoch 9/20
1611/1611 [==============================] - 11s 7ms/step - loss: 233.7947 - val_loss: 295.6

(51538, 40, 5) (51538, 1) (248, 40, 5)
Epoch 1/20
1611/1611 [==============================] - 12s 7ms/step - loss: 994.6793 - val_loss: 1104.8896
Epoch 2/20
1611/1611 [==============================] - 12s 7ms/step - loss: 449.0446 - val_loss: 805.1175
Epoch 3/20
1611/1611 [==============================] - 11s 7ms/step - loss: 391.5176 - val_loss: 606.9626
Epoch 4/20
1611/1611 [==============================] - 12s 7ms/step - loss: 316.0338 - val_loss: 329.8018
Epoch 5/20
1611/1611 [==============================] - 12s 7ms/step - loss: 296.5393 - val_loss: 400.7310
Epoch 6/20
1611/1611 [==============================] - 11s 7ms/step - loss: 278.9533 - val_loss: 327.1832
Epoch 7/20
1611/1611 [==============================] - 11s 7ms/step - loss: 266.5649 - val_loss: 292.1869
Epoch 8/20
1611/1611 [==============================] - 12s 7ms/step - loss: 252.3264 - val_loss: 257.4317
Epoch 9/20
1611/1611 [==============================] - 11s 7ms/step - loss: 240.3982 - val_loss: 239.59

(51538, 40, 5) (51538, 1) (248, 40, 5)
Epoch 1/20
1611/1611 [==============================] - 12s 7ms/step - loss: 1005.1373 - val_loss: 1119.7898
Epoch 2/20
1611/1611 [==============================] - 11s 7ms/step - loss: 440.6535 - val_loss: 666.8292
Epoch 3/20
1611/1611 [==============================] - 11s 7ms/step - loss: 375.2309 - val_loss: 324.8863
Epoch 4/20
1611/1611 [==============================] - 11s 7ms/step - loss: 323.5302 - val_loss: 289.1518
Epoch 5/20
1611/1611 [==============================] - 12s 8ms/step - loss: 293.7157 - val_loss: 264.5674
Epoch 6/20
1611/1611 [==============================] - 11s 7ms/step - loss: 276.4119 - val_loss: 273.2298
Epoch 7/20
1611/1611 [==============================] - 11s 7ms/step - loss: 262.9656 - val_loss: 284.4217
Epoch 8/20
1611/1611 [==============================] - 11s 7ms/step - loss: 248.2249 - val_loss: 258.8352
Epoch 9/20
1611/1611 [==============================] - 11s 7ms/step - loss: 236.3213 - val_loss: 363.0

(51538, 40, 5) (51538, 1) (248, 40, 5)
Epoch 1/20
1611/1611 [==============================] - 13s 8ms/step - loss: 981.6541 - val_loss: 564.4487
Epoch 2/20
1611/1611 [==============================] - 11s 7ms/step - loss: 447.9604 - val_loss: 651.7050
Epoch 3/20
1611/1611 [==============================] - 12s 7ms/step - loss: 383.2815 - val_loss: 370.7835
Epoch 4/20
1611/1611 [==============================] - 11s 7ms/step - loss: 317.7560 - val_loss: 301.7954
Epoch 5/20
1611/1611 [==============================] - 11s 7ms/step - loss: 298.4744 - val_loss: 333.3961
Epoch 6/20
1611/1611 [==============================] - 12s 7ms/step - loss: 281.8705 - val_loss: 318.7209
Epoch 7/20
1611/1611 [==============================] - 12s 7ms/step - loss: 265.3545 - val_loss: 269.8325
Epoch 8/20
1611/1611 [==============================] - 11s 7ms/step - loss: 251.0957 - val_loss: 280.0313
Epoch 9/20
1611/1611 [==============================] - 12s 7ms/step - loss: 237.1234 - val_loss: 290.864

(51538, 40, 5) (51538, 1) (248, 40, 5)
Epoch 1/20
1611/1611 [==============================] - 12s 7ms/step - loss: 1045.3427 - val_loss: 830.9023
Epoch 2/20
1611/1611 [==============================] - 12s 7ms/step - loss: 442.2701 - val_loss: 860.0965
Epoch 3/20
1611/1611 [==============================] - 12s 7ms/step - loss: 385.9459 - val_loss: 685.2007
Epoch 4/20
1611/1611 [==============================] - 11s 7ms/step - loss: 318.3638 - val_loss: 593.9271
Epoch 5/20
1611/1611 [==============================] - 12s 8ms/step - loss: 295.5399 - val_loss: 344.9670
Epoch 6/20
1611/1611 [==============================] - 12s 7ms/step - loss: 275.0363 - val_loss: 422.4583
Epoch 7/20
1611/1611 [==============================] - 12s 7ms/step - loss: 259.7030 - val_loss: 371.8749
Epoch 8/20
1611/1611 [==============================] - 11s 7ms/step - loss: 248.9927 - val_loss: 404.6609
Epoch 9/20
1611/1611 [==============================] - 11s 7ms/step - loss: 240.0711 - val_loss: 468.77

In [5]:
results

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,15.083231,0.0,1416.246871,0.0,227.503845,0.0,[64],0.2,tanh,32,40,225.823854
1,14.917217,0.0,1350.649618,0.0,222.523392,0.0,[64],0.2,tanh,32,40,229.871984
2,19.024605,0.0,2900.514974,0.0,361.935608,0.0,[64],0.2,tanh,32,40,230.356510
3,15.346217,0.0,1048.587734,0.0,235.506348,0.0,[64],0.2,tanh,32,40,237.527807
4,15.655384,0.0,1472.123818,0.0,245.091095,0.0,[64],0.2,tanh,32,40,235.396072


### FD002 <a class="anchor" id="fd002"></a>

In [6]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
remaining_sensors = ["Ps30", "Nf", "phi", "epr"] # selection based on main_notebook [1] "Nf" [1] "Ps30" [1] "phi"
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [7]:
%%time

# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 16.603718	0.0	1.343367e+03	0.0	275.683502	0.0	0.20	[32]	0.1	tanh	128	40
results002 = pd.DataFrame()

for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.2
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [32]
    dropout = 0.1
    activation = 'tanh'
    batch_size = 128
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
       
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results002 = pd.concat([results002, pd.DataFrame(d, index=[0])], ignore_index=True)
    results002.to_csv('results/base/fd002.csv')

(43619, 40, 4) (43619, 1) (259, 40, 4)
Epoch 1/20
341/341 [==============================] - 4s 9ms/step - loss: 2236.6399 - val_loss: 860.0704
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 632.3915 - val_loss: 335.7599
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 446.0885 - val_loss: 323.0456
Epoch 4/20
341/341 [==============================] - 3s 9ms/step - loss: 406.7784 - val_loss: 348.0060
Epoch 5/20
341/341 [==============================] - 3s 9ms/step - loss: 380.7405 - val_loss: 302.4825
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 368.7476 - val_loss: 318.1848
Epoch 7/20
341/341 [==============================] - 3s 9ms/step - loss: 360.7546 - val_loss: 289.3883
Epoch 8/20
341/341 [==============================] - 3s 9ms/step - loss: 352.9236 - val_loss: 283.3387
Epoch 9/20
341/341 [==============================] - 3s 8ms/step - loss: 341.6450 - val_loss: 286.5251
Epoch 10/20
341/341 [===

(43619, 40, 4) (43619, 1) (259, 40, 4)
Epoch 1/20
341/341 [==============================] - 4s 10ms/step - loss: 2412.6331 - val_loss: 1778.6327
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 1067.7266 - val_loss: 372.5059
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 423.9863 - val_loss: 360.1047
Epoch 4/20
341/341 [==============================] - 3s 9ms/step - loss: 390.6329 - val_loss: 385.9352
Epoch 5/20
341/341 [==============================] - 3s 9ms/step - loss: 370.3245 - val_loss: 422.2668
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 360.5595 - val_loss: 429.8504
Epoch 7/20
341/341 [==============================] - 3s 9ms/step - loss: 351.3489 - val_loss: 394.6701
Epoch 8/20
341/341 [==============================] - 3s 9ms/step - loss: 348.6140 - val_loss: 437.7506
Epoch 9/20
341/341 [==============================] - 3s 9ms/step - loss: 342.6008 - val_loss: 416.0578
Epoch 10/20
341/341 [

(43619, 40, 4) (43619, 1) (259, 40, 4)
Epoch 1/20
341/341 [==============================] - 4s 9ms/step - loss: 2348.5066 - val_loss: 1760.0104
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 905.3906 - val_loss: 538.0271
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 402.0147 - val_loss: 497.5231
Epoch 4/20
341/341 [==============================] - 3s 9ms/step - loss: 379.9032 - val_loss: 480.3297
Epoch 5/20
341/341 [==============================] - 3s 8ms/step - loss: 365.4847 - val_loss: 461.1993
Epoch 6/20
341/341 [==============================] - 3s 8ms/step - loss: 358.5267 - val_loss: 442.4182
Epoch 7/20
341/341 [==============================] - 3s 8ms/step - loss: 352.4693 - val_loss: 417.7172
Epoch 8/20
341/341 [==============================] - 3s 9ms/step - loss: 341.7758 - val_loss: 392.9417
Epoch 9/20
341/341 [==============================] - 3s 9ms/step - loss: 336.1290 - val_loss: 361.3725
Epoch 10/20
341/341 [==

(43619, 40, 4) (43619, 1) (259, 40, 4)
Epoch 1/20
341/341 [==============================] - 4s 9ms/step - loss: 2393.1924 - val_loss: 1775.1542
Epoch 2/20
341/341 [==============================] - 3s 8ms/step - loss: 1060.2963 - val_loss: 439.3153
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 429.9979 - val_loss: 386.2940
Epoch 4/20
341/341 [==============================] - 3s 8ms/step - loss: 382.7807 - val_loss: 337.7810
Epoch 5/20
341/341 [==============================] - 3s 9ms/step - loss: 363.5873 - val_loss: 323.6670
Epoch 6/20
341/341 [==============================] - 3s 8ms/step - loss: 350.1389 - val_loss: 327.8008
Epoch 7/20
341/341 [==============================] - 3s 8ms/step - loss: 339.6621 - val_loss: 307.6828
Epoch 8/20
341/341 [==============================] - 3s 8ms/step - loss: 337.8290 - val_loss: 311.1031
Epoch 9/20
341/341 [==============================] - 3s 8ms/step - loss: 336.2205 - val_loss: 346.6529
Epoch 10/20
341/341 [=

(43619, 40, 4) (43619, 1) (259, 40, 4)
Epoch 1/20
341/341 [==============================] - 4s 8ms/step - loss: 2344.3616 - val_loss: 1844.4548
Epoch 2/20
341/341 [==============================] - 3s 8ms/step - loss: 1161.2739 - val_loss: 390.6000
Epoch 3/20
341/341 [==============================] - 3s 8ms/step - loss: 419.1754 - val_loss: 393.2776
Epoch 4/20
341/341 [==============================] - 3s 8ms/step - loss: 391.0252 - val_loss: 378.7570
Epoch 5/20
341/341 [==============================] - 3s 8ms/step - loss: 366.8130 - val_loss: 368.2174
Epoch 6/20
341/341 [==============================] - 3s 8ms/step - loss: 353.4683 - val_loss: 397.9230
Epoch 7/20
341/341 [==============================] - 3s 8ms/step - loss: 345.8092 - val_loss: 369.9594
Epoch 8/20
341/341 [==============================] - 3s 8ms/step - loss: 341.9616 - val_loss: 359.5326
Epoch 9/20
341/341 [==============================] - 3s 8ms/step - loss: 333.7926 - val_loss: 386.5799
Epoch 10/20
341/341 [=

In [8]:
results002

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,16.742281,0.0,1304.559511,0.0,280.303986,0.0,[32],0.1,tanh,128,40,61.363432
1,19.384987,0.0,9952.746447,0.0,375.777710,0.0,[32],0.1,tanh,128,40,62.251372
2,17.638299,0.0,1785.090727,0.0,311.109650,0.0,[32],0.1,tanh,128,40,60.280839
3,17.164997,0.0,2051.300692,0.0,294.637146,0.0,[32],0.1,tanh,128,40,55.732981
4,17.320034,0.0,1426.378111,0.0,299.983521,0.0,[32],0.1,tanh,128,40,55.506745


### FD003 <a class="anchor" id="fd003"></a>

In [9]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD003.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
remaining_sensors = ["Ps30", "NRc", "W31", "BPR"] # selection based on main_notebook [1] "Nf" [1] "Ps30" [1] "phi"
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(24720, 27) (16596, 26) (100, 1)


In [10]:
%%time

# 	RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 43	15.404917	0.0	8.255290e+02	0.0	237.311462	0.0	0.10	[64]	0.2	tanh	32	35
results003 = pd.DataFrame()

for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.1
    sequence_length = 35
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
       
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results003 = pd.concat([results003, pd.DataFrame(d, index=[0])], ignore_index=True)
    results003.to_csv('results/base/fd003.csv')

(21320, 35, 4) (21320, 1) (100, 35, 4)
Epoch 1/20
667/667 [==============================] - 5s 6ms/step - loss: 1809.3916 - val_loss: 627.0729
Epoch 2/20
667/667 [==============================] - 4s 6ms/step - loss: 476.0309 - val_loss: 546.0072
Epoch 3/20
667/667 [==============================] - 4s 7ms/step - loss: 428.4308 - val_loss: 418.3187
Epoch 4/20
667/667 [==============================] - 4s 7ms/step - loss: 397.5644 - val_loss: 534.5612
Epoch 5/20
667/667 [==============================] - 4s 6ms/step - loss: 354.9685 - val_loss: 517.6132
Epoch 6/20
667/667 [==============================] - 4s 6ms/step - loss: 338.8925 - val_loss: 385.0986
Epoch 7/20
667/667 [==============================] - 4s 6ms/step - loss: 328.4911 - val_loss: 380.4737
Epoch 8/20
667/667 [==============================] - 4s 7ms/step - loss: 328.2462 - val_loss: 351.5383
Epoch 9/20
667/667 [==============================] - 4s 6ms/step - loss: 310.2026 - val_loss: 346.5166
Epoch 10/20
667/667 [===

(21320, 35, 4) (21320, 1) (100, 35, 4)
Epoch 1/20
667/667 [==============================] - 5s 6ms/step - loss: 1959.3773 - val_loss: 1284.4646
Epoch 2/20
667/667 [==============================] - 4s 6ms/step - loss: 512.9575 - val_loss: 534.6699
Epoch 3/20
667/667 [==============================] - 4s 6ms/step - loss: 415.3179 - val_loss: 451.7582
Epoch 4/20
667/667 [==============================] - 4s 6ms/step - loss: 368.0338 - val_loss: 504.9758
Epoch 5/20
667/667 [==============================] - 4s 6ms/step - loss: 335.9818 - val_loss: 459.8291
Epoch 6/20
667/667 [==============================] - 4s 6ms/step - loss: 333.8476 - val_loss: 408.7294
Epoch 7/20
667/667 [==============================] - 4s 6ms/step - loss: 323.1582 - val_loss: 516.1066
Epoch 8/20
667/667 [==============================] - 4s 6ms/step - loss: 306.9634 - val_loss: 347.6187
Epoch 9/20
667/667 [==============================] - 4s 6ms/step - loss: 300.6796 - val_loss: 343.1248
Epoch 10/20
667/667 [==

Epoch 1/20
667/667 [==============================] - 5s 6ms/step - loss: 1888.2330 - val_loss: 706.7651
Epoch 2/20
667/667 [==============================] - 4s 6ms/step - loss: 497.0775 - val_loss: 435.4746
Epoch 3/20
667/667 [==============================] - 4s 6ms/step - loss: 419.4802 - val_loss: 654.4635
Epoch 4/20
667/667 [==============================] - 4s 6ms/step - loss: 366.1762 - val_loss: 470.4344
Epoch 5/20
667/667 [==============================] - 4s 6ms/step - loss: 355.1180 - val_loss: 352.7591
Epoch 6/20
667/667 [==============================] - 4s 6ms/step - loss: 336.1967 - val_loss: 471.1773
Epoch 7/20
667/667 [==============================] - 4s 6ms/step - loss: 334.3104 - val_loss: 389.3570
Epoch 8/20
667/667 [==============================] - 4s 6ms/step - loss: 317.4855 - val_loss: 359.4142
Epoch 9/20
667/667 [==============================] - 4s 6ms/step - loss: 311.0001 - val_loss: 346.6952
Epoch 10/20
667/667 [==============================] - 4s 6ms/s

(21320, 35, 4) (21320, 1) (100, 35, 4)
Epoch 1/20
667/667 [==============================] - 5s 7ms/step - loss: 1946.0426 - val_loss: 894.0097
Epoch 2/20
667/667 [==============================] - 4s 6ms/step - loss: 528.4478 - val_loss: 485.9036
Epoch 3/20
667/667 [==============================] - 4s 6ms/step - loss: 454.2690 - val_loss: 753.2277
Epoch 4/20
667/667 [==============================] - 4s 6ms/step - loss: 406.4329 - val_loss: 443.8639
Epoch 5/20
667/667 [==============================] - 4s 6ms/step - loss: 358.9470 - val_loss: 427.8487
Epoch 6/20
667/667 [==============================] - 4s 6ms/step - loss: 339.7233 - val_loss: 359.7825
Epoch 7/20
667/667 [==============================] - 4s 6ms/step - loss: 321.8696 - val_loss: 581.3495
Epoch 8/20
667/667 [==============================] - 4s 6ms/step - loss: 310.6410 - val_loss: 378.3747
Epoch 9/20
667/667 [==============================] - 4s 6ms/step - loss: 295.5874 - val_loss: 308.0919
Epoch 10/20
667/667 [===

Epoch 1/20
667/667 [==============================] - 5s 6ms/step - loss: 1827.8146 - val_loss: 593.7330
Epoch 2/20
667/667 [==============================] - 4s 6ms/step - loss: 487.5656 - val_loss: 481.2301
Epoch 3/20
667/667 [==============================] - 4s 6ms/step - loss: 393.5088 - val_loss: 417.6632
Epoch 4/20
667/667 [==============================] - 4s 6ms/step - loss: 356.6049 - val_loss: 519.1750
Epoch 5/20
667/667 [==============================] - 4s 6ms/step - loss: 336.2125 - val_loss: 419.7113
Epoch 6/20
667/667 [==============================] - 4s 6ms/step - loss: 324.0346 - val_loss: 350.1489
Epoch 7/20
667/667 [==============================] - 4s 6ms/step - loss: 305.5600 - val_loss: 327.7593
Epoch 8/20
667/667 [==============================] - 4s 6ms/step - loss: 290.0201 - val_loss: 361.1849
Epoch 9/20
667/667 [==============================] - 4s 6ms/step - loss: 281.4741 - val_loss: 312.9975
Epoch 10/20
667/667 [==============================] - 4s 6ms/s

In [11]:
results003

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,15.227885,0.0,742.982769,0.0,231.888474,0.0,[64],0.2,tanh,32,35,84.279276
1,15.256113,0.0,1119.886405,0.0,232.748962,0.0,[64],0.2,tanh,32,35,78.741477
2,16.271719,0.0,1376.477150,0.0,264.768829,0.0,[64],0.2,tanh,32,35,77.551248
3,16.100702,0.0,1310.040013,0.0,259.232605,0.0,[64],0.2,tanh,32,35,77.406572
4,15.860561,0.0,1515.197137,0.0,251.557327,0.0,[64],0.2,tanh,32,35,78.349120


### FD001 <a class="anchor" id="fd001"></a>

In [12]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
remaining_sensors = ["Ps30", "Nf", "P30", "BPR"] # selection based on main_notebook 
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(20631, 27) (13096, 26) (100, 1)


In [13]:
%%time

# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 21	13.768992	0.0	3.830708e+02	0.0	189.585159	0.0	0.10	[128]	0.2	tanh	64	30
results001 = pd.DataFrame()

for SEED in range(5):  
    set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.1
    sequence_length = 30
    epochs = 20
    nodes_per_layer = [128]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 64
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
       
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results001 = pd.concat([results001, pd.DataFrame(d, index=[0])], ignore_index=True)
    results001.to_csv('results/base/fd001.csv')

(17731, 30, 4) (17731, 1) (100, 30, 4)
Epoch 1/20
278/278 [==============================] - 5s 16ms/step - loss: 2094.1931 - val_loss: 1618.0125
Epoch 2/20
278/278 [==============================] - 5s 17ms/step - loss: 1216.7513 - val_loss: 701.2263
Epoch 3/20
278/278 [==============================] - 4s 16ms/step - loss: 684.6442 - val_loss: 430.1585
Epoch 4/20
278/278 [==============================] - 5s 16ms/step - loss: 395.6197 - val_loss: 269.1258
Epoch 5/20
278/278 [==============================] - 4s 16ms/step - loss: 333.9962 - val_loss: 234.5949
Epoch 6/20
278/278 [==============================] - 4s 16ms/step - loss: 315.4727 - val_loss: 242.5693
Epoch 7/20
278/278 [==============================] - 4s 16ms/step - loss: 293.2221 - val_loss: 234.5160
Epoch 8/20
278/278 [==============================] - 4s 16ms/step - loss: 289.0195 - val_loss: 240.1270
Epoch 9/20
278/278 [==============================] - 4s 16ms/step - loss: 272.7834 - val_loss: 211.5993
Epoch 10/20
2

(17731, 30, 4) (17731, 1) (100, 30, 4)
Epoch 1/20
278/278 [==============================] - 5s 16ms/step - loss: 2078.9531 - val_loss: 1606.9794
Epoch 2/20
278/278 [==============================] - 4s 16ms/step - loss: 1220.6165 - val_loss: 759.2280
Epoch 3/20
278/278 [==============================] - 5s 16ms/step - loss: 633.4005 - val_loss: 481.1546
Epoch 4/20
278/278 [==============================] - 4s 16ms/step - loss: 432.7019 - val_loss: 517.8420
Epoch 5/20
278/278 [==============================] - 5s 17ms/step - loss: 352.8484 - val_loss: 229.2370
Epoch 6/20
278/278 [==============================] - 4s 16ms/step - loss: 308.6838 - val_loss: 242.9386
Epoch 7/20
278/278 [==============================] - 4s 16ms/step - loss: 286.3264 - val_loss: 496.5279
Epoch 8/20
278/278 [==============================] - 5s 16ms/step - loss: 311.5746 - val_loss: 230.1150
Epoch 9/20
278/278 [==============================] - 4s 16ms/step - loss: 285.4235 - val_loss: 270.9725
Epoch 10/20
2

(17731, 30, 4) (17731, 1) (100, 30, 4)
Epoch 1/20
278/278 [==============================] - 5s 16ms/step - loss: 2094.0459 - val_loss: 1729.9291
Epoch 2/20
278/278 [==============================] - 5s 17ms/step - loss: 1309.5995 - val_loss: 749.9687
Epoch 3/20
278/278 [==============================] - 4s 16ms/step - loss: 747.7368 - val_loss: 693.5231
Epoch 4/20
278/278 [==============================] - 5s 17ms/step - loss: 678.0728 - val_loss: 510.0824
Epoch 5/20
278/278 [==============================] - 5s 17ms/step - loss: 414.9129 - val_loss: 939.8067
Epoch 6/20
278/278 [==============================] - 5s 17ms/step - loss: 363.2160 - val_loss: 341.2820
Epoch 7/20
278/278 [==============================] - 5s 16ms/step - loss: 311.7357 - val_loss: 384.8770
Epoch 8/20
278/278 [==============================] - 5s 17ms/step - loss: 291.5698 - val_loss: 217.9052
Epoch 9/20
278/278 [==============================] - 5s 17ms/step - loss: 277.6761 - val_loss: 237.6449
Epoch 10/20
2

Epoch 1/20
278/278 [==============================] - 5s 15ms/step - loss: 2110.8755 - val_loss: 1565.2759
Epoch 2/20
278/278 [==============================] - 4s 15ms/step - loss: 1210.2472 - val_loss: 749.0840
Epoch 3/20
278/278 [==============================] - 4s 16ms/step - loss: 728.4390 - val_loss: 701.4253
Epoch 4/20
278/278 [==============================] - 4s 16ms/step - loss: 492.8962 - val_loss: 317.3288
Epoch 5/20
278/278 [==============================] - 4s 16ms/step - loss: 348.7446 - val_loss: 308.9222
Epoch 6/20
278/278 [==============================] - 4s 15ms/step - loss: 324.0099 - val_loss: 259.0289
Epoch 7/20
278/278 [==============================] - 4s 15ms/step - loss: 302.8770 - val_loss: 292.0851
Epoch 8/20
278/278 [==============================] - 4s 15ms/step - loss: 294.5700 - val_loss: 341.5771
Epoch 9/20
278/278 [==============================] - 4s 15ms/step - loss: 290.2554 - val_loss: 243.2938
Epoch 10/20
278/278 [==============================]

(17731, 30, 4) (17731, 1) (100, 30, 4)
Epoch 1/20
278/278 [==============================] - 5s 15ms/step - loss: 2124.2063 - val_loss: 1588.0610
Epoch 2/20
278/278 [==============================] - 4s 15ms/step - loss: 1256.3977 - val_loss: 722.7924
Epoch 3/20
278/278 [==============================] - 4s 16ms/step - loss: 738.5599 - val_loss: 756.6016
Epoch 4/20
278/278 [==============================] - 5s 19ms/step - loss: 666.0049 - val_loss: 456.2169
Epoch 5/20
278/278 [==============================] - 4s 15ms/step - loss: 457.7518 - val_loss: 420.1334
Epoch 6/20
278/278 [==============================] - 4s 16ms/step - loss: 352.0724 - val_loss: 252.0757
Epoch 7/20
278/278 [==============================] - 4s 16ms/step - loss: 320.5235 - val_loss: 221.1909
Epoch 8/20
278/278 [==============================] - 4s 16ms/step - loss: 300.2021 - val_loss: 437.0053
Epoch 9/20
278/278 [==============================] - 4s 16ms/step - loss: 299.6741 - val_loss: 216.6460
Epoch 10/20
2

In [14]:
results001

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,13.748364,0.0,356.682616,0.0,189.017532,0.0,[128],0.2,tanh,64,30,88.835431
1,15.438658,0.0,407.824525,0.0,238.352188,0.0,[128],0.2,tanh,64,30,93.002537
2,14.138332,0.0,369.935529,0.0,199.892426,0.0,[128],0.2,tanh,64,30,92.465764
3,14.236129,0.0,403.798504,0.0,202.667404,0.0,[128],0.2,tanh,64,30,87.789842
4,15.131261,0.0,489.257022,0.0,228.955078,0.0,[128],0.2,tanh,64,30,88.103285
